# 공공데이터 상권정보 분석해 보기
* https://www.data.go.kr/dataset/15012005/fileData.do
* 국가중점데이터인 상권정보를 살펴봅니다.
* 처음으로 파일데이터에서 상권별 업종밀집통계정보를 다운로드 받아봅니다.

## !중요
### 버전과 한글폰트
* 한글폰트 사용하기 : http://corazzon.github.io/matplotlib_font_setting

In [ ]:
# 아래의 버전에 의존성이 있습니다. 
# 다음의 버전으로 설치해 주세요.
# !pip install plotnine==0.5.0
# !pip install –upgrade pandas==0.23.4
# !pip install folium==0.5.0
# !pip install statsmodels -U

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
import pandas as pd
from plotnine import *

%matplotlib inline

In [ ]:
%ls data/commercial_store/

# 2016년 상권별 업종밀집 통계
* 상권별 밀집도를 보여준다.

In [ ]:
shop = pd.read_csv('data/commercial_store/상권별_업종밀집통계_2016_.csv', 
                   encoding='cp949', 
                   engine='python')
shop.shape

In [ ]:
shop.head()

In [ ]:
shop.tail()

In [ ]:
shop['대분류명'].value_counts()

In [ ]:
shop['중분류명'].value_counts()

# 상가업소정보 2018년 6월

In [ ]:
%ls data/shop_201806

In [ ]:
# 상가업소정보 2018년 6월 데이터
shop_2018_01 = pd.read_csv('data/shop_201806/shop_201806_01.csv', 
                           encoding='cp949', 
                           engine='python')
shop_2018_01.shape

In [ ]:
shop_2018_01.head(3)

In [ ]:
shop_2018_01.tail(3)

In [ ]:
shop_2018_01.columns

In [ ]:
view_columns = ['상호명', '지점명', '상권업종대분류명', '상권업종중분류명',
              '상권업종소분류명', '시도명', '시군구명', 
              '행정동명', '법정동명', '지번주소', '경도', '위도']
shop_2018_01[view_columns].head()

In [ ]:
# 생각보다 결측치가 많다.
shop_2018_01.isnull().sum()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Window 의 한글 폰트 설정
# plt.rc('font',family='Malgun Gothic')
# Mac 의 한글 폰트 설정
plt.rc('font', family='AppleGothic') 

In [ ]:
import missingno as msno

msno.matrix(shop_2018_01)

In [ ]:
# 모든 데이터를 한번에 찍으면 너무 오래 걸려서 1000개만 찍어봅니다.
# 양쪽에 데이터가 몰려있는데 왜 그런지 주소 정보를 다시보겠습니다.
(ggplot(shop_2018_01[:1000])
 + aes(x='경도', y='위도')
 + geom_point()
 + theme(text=element_text(family='NanumBarunGothic'))
)


In [ ]:
# 데이터를 보니 서울과 부산만 있는거 같다. 
# 그래서 서울과 그 외 데이터로 나눠봤다.
shop_seoul = shop_2018_01.loc[shop_2018_01['도로명주소'].str.startswith('서울')]
shop_except_seoul = shop_2018_01.loc[~shop_2018_01['도로명주소'].str.startswith('서울')]
print(shop_seoul.shape)
print(shop_except_seoul.shape)

In [ ]:
# Pandas 의 plot 으로 위도와 경도를 산점도로 그려봅니다.
shop_seoul.plot.scatter(x='경도', y='위도')

In [ ]:
plt.figure(figsize=(12, 12))
sns.scatterplot(data=shop_seoul, x='경도', y='위도', hue='구군', 
                alpha='auto')

In [ ]:
# 서울만 찍어봤더니 서울지도 비슷하게 나온다
# (ggplot(shop_seoul)
#  + aes(x='경도', y='위도')
#  + geom_point(color='black', alpha=0.2, size=0.2)
#  + theme(text=element_text(family='NanumBarunGothic'))
# )

In [ ]:
shop_2018_01['도로명주소'].head()

In [ ]:
print(shop_2018_01.shape)
shop_2018_01['시도'] = shop_2018_01['도로명주소'].str.split(' ', expand=True)[0]
shop_2018_01['구군'] = shop_2018_01['도로명주소'].str.split(' ', expand=True)[1]
print(shop_2018_01.shape)
shop_2018_01.columns

In [ ]:
# 서울만 찍어봤더니 서울지도 비슷하게 나온다
# 서울 DF을 다시 만들어 준다.
shop_seoul = shop_2018_01.loc[shop_2018_01['도로명주소'].str.startswith('서울')]
shop_seoul.shape

In [ ]:
# (ggplot(shop_seoul)
#  + aes(x='경도', y='위도', color='구군')
#  + geom_point(alpha=0.2, size=0.2)
#  + theme(text=element_text(family='NanumBarunGothic'))
#  + scale_fill_gradient(low = 'blue', high = 'green') 
# )

In [ ]:
shop_seoul['상권업종대분류명'].value_counts()

In [ ]:
(ggplot(shop_seoul)
 + aes(x='경도', y='위도', color='상권업종대분류명')
 + geom_point(alpha=0.2, size=0.2)
 + theme(text=element_text(family='NanumBarunGothic'))
 + scale_fill_gradient(low = 'blue', high = 'green') 
)

## 학문/교육과 관련된 정보 보기

In [ ]:
shop_seoul_edu = shop_seoul[shop_seoul['상권업종대분류명'] == '학문/교육']

(ggplot(shop_seoul_edu)
 + aes(x='경도', y='위도', color='상권업종중분류명')
 + geom_point(size=0.2)
 + theme(text=element_text(family='NanumBarunGothic'))
 + scale_fill_gradient(low = 'blue', high = 'green') 
)

## 부동산과 관련된 정보 보기

In [ ]:
shop_seoul_edu = shop_seoul[shop_seoul['상권업종대분류명'] == '부동산']

(ggplot(shop_seoul_edu)
 + aes(x='경도', y='위도', color='상권업종중분류명')
 + geom_point(size=0.2)
 + theme(text=element_text(family='NanumBarunGothic'))
 + scale_fill_gradient(low = 'blue', high = 'green') 
)

### 학원-컴퓨터

In [ ]:
shop_seoul_edu = shop_seoul.loc[shop_seoul['상권업종중분류명'] == '학원-컴퓨터']

# (ggplot(shop_seoul_edu)
#  + aes(x='경도', y='위도', color='상권업종중분류명')
#  + geom_point()
#  + theme(text=element_text(family='NanumBarunGothic'))
#  + scale_fill_gradient(low = 'blue', high = 'green') 
# )

plt.figure(figsize=(12, 12))
sns.scatterplot(data=shop_seoul_edu, x='경도', y='위도', hue='상권업종중분류명', 
                alpha='auto')

In [ ]:
import folium

def show_marker_map(geo_df):
    map = folium.Map(location=[geo_df['위도'].mean(), geo_df['경도'].mean()], 
                     zoom_start=12, tiles='Stamen Terrain')

    for n in geo_df.index:
        shop_name = geo_df.loc[n, '상호명'] + ' - ' + geo_df.loc[n, '도로명주소']
        folium.Marker([geo_df.loc[n, '위도'], 
                       geo_df.loc[n, '경도']], 
                      popup=shop_name).add_to(map)

    return map

In [ ]:
show_marker_map(shop_seoul_edu)

#### 편의점 카페 분포 보기

In [ ]:
shop_seoul_eat = shop_seoul[shop_seoul['상권업종중분류명'] == '커피점/카페']

(ggplot(shop_seoul_eat)
 + aes(x='경도', y='위도', color='구군')
 + geom_point(size=0.7)
 + theme(text=element_text(family='NanumBarunGothic'))
 + scale_fill_gradient(low = 'blue', high = 'green') 
)

In [ ]:
shop_seoul_eat.shape

In [ ]:
geo_df = shop_seoul_eat[shop_seoul_eat["구군"] == "중구"]

map = folium.Map(location=[geo_df['위도'].mean(), 
                           geo_df['경도'].mean()], 
                 zoom_start=16, tiles='Stamen Terrain')

for n in geo_df.index:
    shop_name = geo_df.loc[n, '상호명'] + ' - ' + geo_df.loc[n, '도로명주소']
    folium.Marker([geo_df.loc[n, '위도'], geo_df.loc[n, '경도']], 
                  popup=shop_name).add_to(map)

map

## 서울 외 지역 보기

In [ ]:
# 부산만 있는거 같습니다.
# 서울만 제외한 나머지 지역으로 DF을 다시 생성해 줍니다.
shop_except_seoul = shop_2018_01.loc[~shop_2018_01['도로명주소'].str.startswith('서울')]
(ggplot(shop_except_seoul)
 + aes(x='경도', y='위도', color='구군')
 + geom_point(size=0.7)
 + theme(text=element_text(family='NanumBarunGothic'))
)